## Backend Intern Coding Projects
Name: Guohao Ma  
Tel: 5146550688  
Mail: g7ma@uwaterloo.ca  

*This notebook is written for backend intern coding projects and needs to be opened in Jupyter Notebook.*
   

1. Step 0: Generate Fake Data  
2. Step 1: Write a RESTful API
3. Step 2: Wrtie a Test case
4. Step 3: Write a SAVE2CSV function
5. Step 4: Record actions and result to MySQL DB


### Table of Contents
* [0 Generate Fake Data](#chapter0)  
    * [0.1 Generate Random Worker Info](#chapter0.1)  
    * [0.2 Generate Credentials](#chapter0.2)
* [1 Build a Flask API](#chapter1)  
* [2 Write Test Cases](#chapter2)
* [3 Write a SAVE2CSV Function](#chapter3)
* [4 Record Actions and Result to MySQL DB](#chapter4)
   * [4.1 Create a MySQL DB and Record Table ](#chapter4.1) 
   * [4.2 Fake Data Insertion ](#chapter4.2)
   * [4.3 Test DB Insertion Result ](#chapter4.3)


## Step 0 Generate Fake Data <a class="anchor" id= "chapter0"></a>

### 0.1 Generate Random Worker Info<a class="anchor" id= "chapter0.1"></a>

In [1]:
import random
from random import randint

def generate_random_worker_info():
    """
    randomly generate fake data of 6 workers
    
    """
    worker_status_ls = []
    worker_name_ls = [
        "host p1_pc",
        "host p2_pc",
        "host p3_pc",
        "host p1_01",
        "host p2_00",
        "host filecoin",
    ]
    gpu_ls = ["GeForce RTX 2070", "GeForce RTX 2080 Ti", "GeForce RTX 1080 Ti"," "] # a worker withous a pre-installed GPU is assigned as "". 
    gpu_status_ls = ["not used", "used"]
    vmen_unit = ['GiB', 'TiB']  # In the hardcoded data,vmem could be expressed in GiB and TiB. 
 
    for i in range(0, 6):
        # worker
        single_worker_dict = {}
        single_worker_dict["worker_id"] = i
        single_worker_dict["worker_name"] = worker_name_ls[i]
        # cpu
        cpu_core_int=randint(0, 48)
        single_worker_dict["cpu_core"] = str(cpu_core_int)+ ' core(s)'
        single_worker_dict["cpu_core_usage"] = str(randint(0, cpu_core_int))+ ' core(s) in use'
        # ram
        ram_total_int=round(random.uniform(100, 1000), 2)
        ram_usage_int=round(
            random.uniform(0, ram_total_int), 2
        )
        single_worker_dict['ram_total']= str(ram_total_int) +' GiB'
        single_worker_dict['ram_usage']= str(ram_usage_int)+' GiB'
        single_worker_dict['ram_usage_percentage']=  str(round(100* ram_usage_int/ram_total_int,2))+" %"
        #vmem
        vmem_unit_level=vmen_unit[randint(0,1)]
        if vmem_unit_level=='GiB': # GiB level
            vmem_total_int=round(random.uniform(100, 1000), 2)
            vmem_usage_int=round(
                random.uniform(0,  vmem_total_int), 2
            )
            single_worker_dict['vmem_total']=str(vmem_total_int)+' GiB'
            single_worker_dict['vmem_usage']=str(vmem_usage_int)+' GiB'
            single_worker_dict['vmem_usage_percentage']= str(round(100* vmem_usage_int/vmem_total_int,2))+" %"
        else:# TiB level
            vmem_total_int=round(random.uniform(0, 10), 2)
            vmem_usage_int=round(
                random.uniform(0, vmem_total_int*1024), 2
            )
            single_worker_dict['vmem_total']=str(vmem_total_int)+' TiB'
            if vmem_usage_int>1024:
                single_worker_dict['vmem_usage']=str(round(vmem_usage_int/1024,2))+ ' TiB'
            else:
                single_worker_dict['vmem_usage']=str(vmem_usage_int)+ ' GiB'       
                                                     
            single_worker_dict['vmem_usage_percentage']= str(round(100* vmem_usage_int/(vmem_total_int*1024),2))+" %"
        # gpu
        single_worker_dict['gpu']=gpu_ls[randint(0, 3)]
        if single_worker_dict['gpu']==" ":
            single_worker_dict['gpu_status']=" "
        else:
            single_worker_dict['gpu_status']=gpu_status_ls[randint(0, 1)]
        worker_status_ls.append(single_worker_dict)
    return worker_status_ls

In [2]:
# show the generated fake data
worker_status_ls=generate_random_worker_info()
worker_status_ls

[{'worker_id': 0,
  'worker_name': 'host p1_pc',
  'cpu_core': '39 core(s)',
  'cpu_core_usage': '20 core(s) in use',
  'ram_total': '681.41 GiB',
  'ram_usage': '174.83 GiB',
  'ram_usage_percentage': '25.66 %',
  'vmem_total': '7.88 TiB',
  'vmem_usage': '4.5 TiB',
  'vmem_usage_percentage': '57.05 %',
  'gpu': 'GeForce RTX 2080 Ti',
  'gpu_status': 'used'},
 {'worker_id': 1,
  'worker_name': 'host p2_pc',
  'cpu_core': '28 core(s)',
  'cpu_core_usage': '6 core(s) in use',
  'ram_total': '916.14 GiB',
  'ram_usage': '526.81 GiB',
  'ram_usage_percentage': '57.5 %',
  'vmem_total': '485.43 GiB',
  'vmem_usage': '226.96 GiB',
  'vmem_usage_percentage': '46.75 %',
  'gpu': 'GeForce RTX 2080 Ti',
  'gpu_status': 'not used'},
 {'worker_id': 2,
  'worker_name': 'host p3_pc',
  'cpu_core': '27 core(s)',
  'cpu_core_usage': '2 core(s) in use',
  'ram_total': '914.64 GiB',
  'ram_usage': '30.14 GiB',
  'ram_usage_percentage': '3.3 %',
  'vmem_total': '394.24 GiB',
  'vmem_usage': '263.8 GiB',

### 0.2 Generate Credentials<a class="anchor" id= "chapter0.2"></a>

In [3]:
# generate true credentials
cred_ls=[]
for i in range(0, 6):
    admin_dict={}
    admin_dict['admin']="admin_"+str(i)
    admin_dict['email']=admin_dict['admin']+"@mail.ca"
    admin_dict['pwd']=randint(1000000, 4000000)
    cred_ls.append(admin_dict)
cred_ls   

[{'admin': 'admin_0', 'email': 'admin_0@mail.ca', 'pwd': 3006366},
 {'admin': 'admin_1', 'email': 'admin_1@mail.ca', 'pwd': 1111335},
 {'admin': 'admin_2', 'email': 'admin_2@mail.ca', 'pwd': 3438356},
 {'admin': 'admin_3', 'email': 'admin_3@mail.ca', 'pwd': 1057639},
 {'admin': 'admin_4', 'email': 'admin_4@mail.ca', 'pwd': 2468425},
 {'admin': 'admin_5', 'email': 'admin_5@mail.ca', 'pwd': 3411982}]

## Step1 Build a Flask API<a class="anchor" id= "chapter1"></a>

In [4]:
"""
Step1: Write a Restful API

"""

from flask import Flask, request, jsonify
import json
app = Flask(__name__)

class APIAuthError(Exception):
    """Custom Authentication Error Class."""
    code = 777
    description = "Authentication Error"

@app.route('/get_worker_status/v1', methods=['POST'])
def worker():
    """api request for admin,email and password for authenticaiton"""
    admin= request.json.get('admin')
    email = request.json.get('email')
    password=request.json.get('pwd')
    cred_check={"admin":admin,'email':email,'pwd':password }
    if cred_check in cred_ls:
        return jsonify(generate_random_worker_info())
    else:
        raise APIAuthError('Incorrect Credentials')


In [5]:
## show successfully running result
response =app.test_client().post(
'/get_worker_status/v1',
content_type='application/json',
data=json.dumps(cred_ls[randint(0, 5)])# use true creds
 )
json.loads(response.get_data(as_text=True))

[{'cpu_core': '16 core(s)',
  'cpu_core_usage': '16 core(s) in use',
  'gpu': 'GeForce RTX 1080 Ti',
  'gpu_status': 'not used',
  'ram_total': '807.53 GiB',
  'ram_usage': '558.04 GiB',
  'ram_usage_percentage': '69.1 %',
  'vmem_total': '443.79 GiB',
  'vmem_usage': '415.44 GiB',
  'vmem_usage_percentage': '93.61 %',
  'worker_id': 0,
  'worker_name': 'host p1_pc'},
 {'cpu_core': '38 core(s)',
  'cpu_core_usage': '35 core(s) in use',
  'gpu': 'GeForce RTX 2080 Ti',
  'gpu_status': 'used',
  'ram_total': '576.46 GiB',
  'ram_usage': '98.12 GiB',
  'ram_usage_percentage': '17.02 %',
  'vmem_total': '3.81 TiB',
  'vmem_usage': '717.38 GiB',
  'vmem_usage_percentage': '18.39 %',
  'worker_id': 1,
  'worker_name': 'host p2_pc'},
 {'cpu_core': '0 core(s)',
  'cpu_core_usage': '0 core(s) in use',
  'gpu': 'GeForce RTX 1080 Ti',
  'gpu_status': 'used',
  'ram_total': '438.95 GiB',
  'ram_usage': '16.85 GiB',
  'ram_usage_percentage': '3.84 %',
  'vmem_total': '332.32 GiB',
  'vmem_usage': '7

In [6]:
## show error result
wrong_cred={'admin': 'admin_0', 'email': 'admin_0@mail.ca', 'pwd': 111111} # pwd is wrong --> incorrect credientials
response =app.test_client().post(
'/get_worker_status/v1',
content_type='application/json',
data=json.dumps(wrong_cred)# use wrong creds
 )

[2021-03-14 16:59:20,344] ERROR in app: Exception on /get_worker_status/v1 [POST]
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "<ipython-input-4-7f248eb82f58>", line 25, in worker
    raise

## Step2 Write Test Cases<a class="anchor" id= "chapter2"></a>

In [7]:
"""
Step 2 write unit test case and save the report into .txt file

"""

import unittest
from datetime import datetime

# unittest
class WorkerApiTest(unittest.TestCase):

    def test_result(self):
        response =app.test_client().post(
        '/get_worker_status/v1',
        content_type='application/json',
        data=json.dumps(cred_ls[randint(0, 5)])# use true creds
         )
        self.assertEqual(response.status, '200 OK') # check the response status
        self.assertLessEqual(1,len(response.json)) # return at least one item
        
if __name__ == "__main__":
    suite=unittest.TestLoader().loadTestsFromTestCase(WorkerApiTest)
    result=unittest.TextTestRunner(verbosity=2).run(suite)
    report_output=str(datetime.now()) +"-----------"+ str(result)
    print(report_output)
    
    # write into a txt file
    with open("api_test_report.txt", "a") as text_file:
        text_file.write("\n")
        text_file.write(report_output)

test_result (__main__.WorkerApiTest) ... 

2021-03-14 16:59:20.386339-----------<unittest.runner.TextTestResult run=1 errors=0 failures=0>


ok

----------------------------------------------------------------------
Ran 1 test in 0.006s

OK


## Step3: Write a Save2CSV function<a class="anchor" id= "chapter3"></a>

In [8]:
import pandas as pd
import os.path

def save2csv():
    """
    save api info into a csv file using pandas dataframe
    """
    api_info_all_df=pd.DataFrame(columns = ['api_call_data','api_key','api_values'])    
    if not os.path.isfile('./api_call_info.csv'):    #create a csv if not existed   
        api_info_all_df.to_csv('./api_call_info.csv')
    else:
        print('api_call_info.csv exists, append new info to it')
    api_key_value=cred_ls[randint(0,5)]
    response = app.test_client().post(
        '/get_worker_status/v1',
        content_type='application/json',
        data=json.dumps(api_key_value)
    )   
    api_info_all_df['api_call_data']=[json.loads(response.get_data(as_text=True))]
    api_info_all_df['api_key']=[list(api_key_value.keys())]
    api_info_all_df['api_values']=[list(api_key_value.values())]
    api_info_all_df=api_info_all_df.reset_index(drop=True)
    api_info_all_df.to_csv('./api_call_info.csv',mode='a', header=False)


In [9]:
# save 6 record into record table and show
for i in range(0,6):
    save2csv()
test_result=pd.read_csv('./api_call_info.csv')[['api_call_data','api_key','api_values']]
test_result

api_call_info.csv exists, append new info to it
api_call_info.csv exists, append new info to it
api_call_info.csv exists, append new info to it
api_call_info.csv exists, append new info to it
api_call_info.csv exists, append new info to it
api_call_info.csv exists, append new info to it


,api_call_data,api_key,api_values
0,"[{'cpu_core': 22, 'cpu_core_usage': 5, 'cpu_to...","['admin', 'email', 'pwd']","['admin_4', 'admin_4@aaa.ca', 3249594]"
1,"[{'cpu_core': 34, 'cpu_core_usage': 28, 'cpu_t...","['admin', 'email', 'pwd']","['admin_4', 'admin_4@aaa.ca', 3249594]"
2,"[{'cpu_core': 43, 'cpu_core_usage': 37, 'cpu_t...","['admin', 'email', 'pwd']","['admin_5', 'admin_5@aaa.ca', 2535725]"
3,"[{'cpu_core': 16, 'cpu_core_usage': 5, 'cpu_to...","['admin', 'email', 'pwd']","['admin_3', 'admin_3@aaa.ca', 2270289]"
4,"[{'cpu_core': 47, 'cpu_core_usage': 39, 'cpu_t...","['admin', 'email', 'pwd']","['admin_4', 'admin_4@aaa.ca', 3249594]"
5,"[{'cpu_core': 9, 'cpu_core_usage': 5, 'cpu_tot...","['admin', 'email', 'pwd']","['admin_2', 'admin_2@aaa.ca', 1817693]"
6,"[{'cpu_core': 23, 'cpu_core_usage': 4, 'cpu_to...","['admin', 'email', 'pwd']","['admin_3', 'admin_3@aaa.ca', 1467188]"
7,"[{'cpu_core': 12, 'cpu_core_usage': 2, 'cpu_to...","['admin', 'email', 'pwd']","['admin_0', 'admin_0@aaa.ca', 1799346]"
8,"[{'cpu_core': 19, 'cpu_core_usage': 5, 'cpu_to...","['admin', 'email', 'pwd']","['admin_4', 'admin_4@aaa.ca', 1767844]"
9,"[{'cpu_core': 25, 'cpu_core_usage': 25, 'cpu_t...","['admin', 'email', 'pwd']","['admin_5', 'admin_5@aaa.ca', 3128185]"


## Step4: to MySQL DB <a class="anchor" id= "chapter4"></a>

### Create MySQL DB and Table <a class="anchor" id= "chapter4.1"></a>

In [10]:
import mysql.connector
##------------------ db_schema------------------------##
db_schema="""

CREATE DATABASE IF NOT EXISTS api_database;
USE api_database;
CREATE TABLE IF NOT EXISTS record (
admin_id VARCHAR(255) NOT NULL, 
call_date VARCHAR(255) NOT NULL, 
call_time VARCHAR(255) NOT NULL,
result varchar(25) NOT NULL CHECK (result IN ('success','failed')),
error_counts int,
PRIMARY KEY (admin_id,call_date,call_time));

"""

# using localhost for simplication
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password=""
)
mycursor = mydb.cursor()
mycursor.execute(db_schema)
mydb.close() # close the db

### Fake Data Insertion <a class="anchor" id= "chapter4.2"></a>

In [11]:
import random
from random import randint
from datetime import datetime
import time


admin_data_ls = []
for i in range(0, 6):
    admin_dict= {}
    response = app.test_client().post(
        '/get_worker_status/v1',
        content_type='application/json',
        data= json.dumps(cred_ls[randint(0,5)])
    )

    if response.status=='200 OK':
        admin_dict["result"]="success"
        admin_dict["error_counts"]=0
    else:
        admin_dict["result"]="failed" 
        admin_dict["error_counts"]=randint(0, 8)
    admin_dict["admin"] = "admin_"+ str(randint(0, 5))
    admin_dict["call_date"] =datetime.today().strftime('%Y-%m-%d')
    admin_dict["call_time"] = str(datetime.now().time())
    admin_data_ls.append(admin_dict)
    
    # connect to db
    mydb = mysql.connector.connect(
      host="localhost",
      user="root",
      password="",
      database = 'api_database'
    )
    mycursor = mydb.cursor()
    # start insertion
    sql = "INSERT INTO record (admin_id, call_date,call_time,result,error_counts ) VALUES (%s, %s, %s, %s, %s)"
    val = (admin_dict["admin"],admin_dict["call_date"],admin_dict["call_time"],admin_dict["result"],admin_dict["error_counts"])
    mycursor.execute(sql,val)
    mydb.commit()
    mydb.close()
    time.sleep(1) 
admin_data_ls

[{'result': 'success',
  'error_counts': 0,
  'admin': 'admin_5',
  'call_date': '2021-03-14',
  'call_time': '16:59:20.980456'},
 {'result': 'success',
  'error_counts': 0,
  'admin': 'admin_0',
  'call_date': '2021-03-14',
  'call_time': '16:59:22.025959'},
 {'result': 'success',
  'error_counts': 0,
  'admin': 'admin_4',
  'call_date': '2021-03-14',
  'call_time': '16:59:23.051026'},
 {'result': 'success',
  'error_counts': 0,
  'admin': 'admin_4',
  'call_date': '2021-03-14',
  'call_time': '16:59:24.076179'},
 {'result': 'success',
  'error_counts': 0,
  'admin': 'admin_5',
  'call_date': '2021-03-14',
  'call_time': '16:59:25.099951'},
 {'result': 'success',
  'error_counts': 0,
  'admin': 'admin_3',
  'call_date': '2021-03-14',
  'call_time': '16:59:26.130548'}]

## Test db insertion result <a class="anchor" id= "chapter4.3"></a>

In [12]:
table_content_ls=[]
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    database="api_database"
)
mycursor = mydb.cursor()
sql = "SELECT * FROM api_database.record"  # return all the records
mycursor.execute(sql)
result = mycursor.fetchall() 

# loop through the rows 
for row in result: 
    table_content_ls.append(row)

table_content_ls

[('admin_0', '2021-03-14', '16:59:22.025959', 'success', 0),
 ('admin_3', '2021-03-14', '16:59:26.130548', 'success', 0),
 ('admin_4', '2021-03-14', '16:59:23.051026', 'success', 0),
 ('admin_4', '2021-03-14', '16:59:24.076179', 'success', 0),
 ('admin_5', '2021-03-14', '16:59:20.980456', 'success', 0),
 ('admin_5', '2021-03-14', '16:59:25.099951', 'success', 0)]